In [4]:
%reload_ext autoreload
%autoreload 2

### Here, we load our full historical data into Hopsworks to initiate (backfill) the data into there. First, get this full historical data into a pandas dataframe

In [4]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath('../'))
from src.load_transform_data import transform_ohlc_to_features_target

features_and_target = transform_ohlc_to_features_target()
features_and_target

,date,open,high,low,close,volume,volume_eth,market_cap,tommorow_price,target,...,sma_10,ema_25,rsi_25,sma_25,ema_50,rsi_50,sma_50,ema_100,rsi_100,sma_100
3031,2015-11-15,0.8912,0.9215,0.8750,0.9064,4.118000e+05,458365.0,6.720000e+07,0.9290,1,...,0.984672,1.040036,50.958458,1.024686,1.049078,46.417700,1.216422,0.942045,43.066211,0.959506
3030,2015-11-16,0.9062,0.9447,0.8920,0.9290,6.209000e+05,676442.0,6.860000e+07,1.0100,1,...,1.008894,1.060586,51.682152,1.033352,1.072073,46.821823,1.239824,0.966193,43.328367,0.981604
3029,2015-11-17,0.9249,1.0300,0.9058,1.0100,1.100000e+06,1183690.0,7.220000e+07,0.9900,0,...,1.087179,1.139641,54.205128,1.100424,1.158029,48.249270,1.332732,1.049388,44.257583,1.063728
3028,2015-11-18,0.9900,1.0100,0.9405,0.9900,6.811000e+05,691994.0,7.360000e+07,0.9555,0,...,1.070259,1.107104,53.486795,1.058955,1.129116,47.925148,1.295100,1.028025,44.077332,1.039581
3027,2015-11-19,0.9887,1.0100,0.9375,0.9555,4.435000e+05,455866.0,7.280000e+07,0.9249,0,...,1.037955,1.062920,52.242769,1.007418,1.085945,47.365082,1.242914,0.992353,43.766743,1.004561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2024-03-17,3533.0000,3675.0000,3428.0000,3645.0000,1.820000e+10,5080870.0,4.382000e+11,3525.0000,0,...,0.946434,1.027431,61.180844,1.025305,1.133170,62.998705,1.194471,1.286674,61.697488,1.354072
4,2024-03-18,3641.0000,3641.0000,3469.0000,3525.0000,1.870000e+10,5241367.0,4.361000e+11,3160.0000,0,...,0.923912,0.994095,57.486834,0.985408,1.091760,60.652518,1.145626,1.238324,60.264560,1.303750
3,2024-03-19,3521.0000,3544.0000,3152.0000,3160.0000,2.930000e+10,8830574.0,4.062000e+11,3512.0000,1,...,0.844784,0.898685,48.255406,0.881047,0.979530,54.368158,1.021391,1.107685,56.250710,1.165265
2,2024-03-20,3166.0000,3528.0000,3067.0000,3512.0000,3.390000e+10,10308565.0,4.023000e+11,3491.0000,0,...,0.948215,0.998884,55.443163,0.973544,1.084871,58.590346,1.126667,1.225466,58.916242,1.288949


In [6]:
sum(features_and_target['volume_eth'] < 0)

2

### Create the objects necessary to initate a connection to our hopsworks feature store. A feature group is like a specific "dataframe" within a feature store. There can be multiple feature groups in a feature store.

In [7]:
import os
from dotenv import load_dotenv
import hopsworks

load_dotenv('../.env')

HOPSWORKS_PROJECT_NAME = os.environ['HOPSWORKS_PROJECT_NAME']
HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
FEATURE_GROUP_NAME = 'ethereum_ohlc_feature_group'
FEATURE_GROUP_VERSION = 1

project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

feature_store = project.get_feature_store()

feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Historical daily ethereum OHLC time series",
    # primary_key = ['date'],
    # event_time='date',
)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.


### Add the OHLC Ethereum data into our feature group. This data has already been processed into predictor and response variables and is ready for machine learning tasks.

In [10]:
feature_group.insert(features_and_target, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 3031/3031 | Elapsed Time: 00:03 | Remaining Time: 00:00

Launching job: ethereum_ohlc_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/582805/jobs/named/ethereum_ohlc_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x133552140>, None)